In [1]:
import geopandas as gpd 
import pandas as pd
from shapely import ops
import os
import shapely
import numpy as np
import psycopg2
import networkx as nx
import multiprocessing as mp
from shapely.geometry import LineString
import pandana as pdna
from pandana import Network
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
shapely.speedups.enable()

%matplotlib inline 

In [3]:
# get cencus tract
con = psycopg2.connect(database="walkability", user="postgres", password=1234,
    host="172.19.118.205",port="5432")
ct = gpd.read_postgis('SELECT * FROM walkability.public.census_tracts_blank',con)

In [4]:
ct.head(3)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,ctnum,geotext,geom
0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100.0,535000100,(POLYGON ((320588.9306980086 4835855.647789237...
1,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200.0,535000200,"(POLYGON ((316642.831678931 4832470.554474635,..."
2,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400.0,535026400,(POLYGON ((314351.8668838083 4845056.100542194...


In [ ]:
#import csv
#import requests

#CSV_URL = 'https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/Table?TableId=2.2.11&GeographyId=2270&GeographyTypeId=3&DisplayAs=Table&GeograghyName=Toronto#csv-export'


#with requests.Session() as s:
#    download = s.get(CSV_URL)

#    decoded_content = download.content.decode('utf-8')

#    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
#    my_list = list(cr)
#    for row in my_list:
 #       print(row)

In [48]:

#df = pd.read_csv('/media/DATADRIVE/realestate/rental_prices/census_tracts/bedr_type_2010.csv')
#ftp://asulaim@172.19.118.205/media/DATADRIVE/walkability/ablajan/str_size_2010.csv
#df = pd.read_csv('/media/DATADRIVE/walkability/ablajan/str_size_2010.csv')
df = pd.read_csv('/media/DATADRIVE/realestate/rental_prices/census_tracts/bedr_type_2017.csv', delimiter= ",", skiprows=2)

In [49]:
df.dtypes

Unnamed: 0      object
Bachelor        object
Unnamed: 2      object
1 Bedroom       object
Unnamed: 4      object
2 Bedroom       object
Unnamed: 6      object
3 Bedroom +     object
Unnamed: 8      object
Total           object
Unnamed: 10     object
Unnamed: 11    float64
dtype: object

In [46]:
df.head()




,Unnamed: 0,Bachelor,Unnamed: 2,1 Bedroom,Unnamed: 4,2 Bedroom,Unnamed: 6,3 Bedroom +,Unnamed: 8,Total,Unnamed: 10,Unnamed: 11
0,0001.00,**,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
1,0002.00,**,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
2,0004.00,880,a,"1,009",b,"1,121",b,**,NaN,982,a,NaN
3,0005.00,900,a,"1,106",c,"1,456",c,**,NaN,"1,088",c,NaN
4,0007.01,938,c,"1,162",b,**,NaN,**,NaN,"1,165",c,NaN


In [50]:
df.tail(10)

,Unnamed: 0,Bachelor,Unnamed: 2,1 Bedroom,Unnamed: 4,2 Bedroom,Unnamed: 6,3 Bedroom +,Unnamed: 8,Total,Unnamed: 10,Unnamed: 11
626,0831.02,**,NaN,"1,145",b,"1,476",b,**,NaN,"1,322",b,NaN
627,Toronto,"1,013",a,"1,194",a,"1,403",a,"1,589",a,"1,308",a,NaN
628,Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629,The following letter codes are used to indicat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630,** Data suppressed to protect confidentiality ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,++ Change in rent is not statistically signifi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
632,- No units exist in the universe for this cat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
633,n/a: Not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
634,"CMA, CA and CSD definitions are based on 2016 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,Source,CMHC Rental Market Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.tail(10)

,Unnamed: 0,Bachelor,Unnamed: 2,1 Bedroom,Unnamed: 4,2 Bedroom,Unnamed: 6,3 Bedroom +,Unnamed: 8,Total,Unnamed: 10,Unnamed: 11
625,0831.01,**,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
626,0831.02,**,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
628,Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629,The following letter codes are used to indicat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630,** Data suppressed to protect confidentiality ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,++ Change in rent is not statistically signifi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
632,- No units exist in the universe for this cat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
633,n/a: Not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
634,"CMA, CA and CSD definitions are based on 2016 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,Source,CMHC Rental Market Survey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
#df.drop(df.index[0], inplace=True)
#df.drop(['Toronto', 'Notes', 'The following letter codes are used to indicate the reliability of the estimates: a - Excellent, b- Very good, c - Good, d - Fair (Use with Caution)', '** Data suppressed to protect confidentiality or data not statistically reliable', '++ Change in rent is not statistically significant. This means that the change in rent is not statistically different than zero (0).  (Applies only to % Change of Average Rent Tables).', ' - No units exist in the universe for this category', 'n/a: Not applicable', 'CMA, CA and CSD definitions are based on 2006 Census Geography Definitions', 'Source'])
#df.drop(df.index[627], inplace=True)
#df.drop(df.index[627], inplace=True)
df.drop(df.tail(9).index,inplace=True)
df.columns

Index(['Unnamed: 0', 'Bachelor', 'Unnamed: 2', '1 Bedroom', 'Unnamed: 4',
       '2 Bedroom', 'Unnamed: 6', '3 Bedroom +', 'Unnamed: 8', 'Total',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [58]:
df = df.rename(columns={'Unnamed: 0': 'ctname', 'Unnamed: 2': 'reliability_ba', '1 Bedroom': 'bedroom_1', 'Unnamed: 4': 'reliability_b1', '2 Bedroom': 'bedroom_2', 'Unnamed: 6': 'reliablity_b2', '3 Bedroom +': 'bedroom_3', 'Unnamed: 10': 'reliability_t' })
df.columns

Index(['ctname', 'Bachelor', 'reliability_ba', 'bedroom_1', 'reliability_b1',
       'bedroom_2', 'reliablity_b2', 'bedroom_3', 'Unnamed: 8', 'Total',
       'reliability_t', 'Unnamed: 11'],
      dtype='object')

In [59]:
ct.merge(df, left_on='ctname', right_on='ctname')

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,...,reliability_ba,bedroom_1,reliability_b1,bedroom_2,reliablity_b2,bedroom_3,Unnamed: 8,Total,reliability_t,Unnamed: 11
0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
1,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
2,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
3,6,217,5350163.00,0163.00,35,Ontario,535,35535,Toronto,B,...,a,907,a,"1,006",a,**,NaN,"1,027",b,NaN
4,7,443,5350213.02,0213.02,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
5,8,455,5350376.04,0376.04,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
6,10,569,5350225.02,0225.02,35,Ontario,535,35535,Toronto,B,...,b,"1,327",b,"1,581",b,**,NaN,"1,432",b,NaN
7,11,1,5350128.04,0128.04,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
8,12,5,5350363.06,0363.06,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
9,16,6,5350010.01,0010.01,35,Ontario,535,35535,Toronto,B,...,NaN,**,NaN,**,NaN,**,NaN,**,NaN,NaN
